In [ ]:
import rtde_control
import sys
import rtde.csv_reader as csv_reader
import time
import pandas as pd
import numpy as np
import math
rtde_c = rtde_control.RTDEControlInterface("127.0.0.1")# Parameters


In [ ]:
#home
joint_q = [0,-1.57,1.57,0,1.57,3.14]# Move to initial joint position with a regular moveJ
rtde_c.moveJ(joint_q)# Execute 500Hz control loop for 2 seconds, each cycle is 2ms


In [ ]:
with open('robot_data_circular_variable_diameter.csv') as csvfile:
    r = csv_reader.CSVReader(csvfile)
    joint_values=[r.target_q_0,r.target_q_1,r.target_q_2,r.target_q_3,r.target_q_4,r.target_q_5]  
    joint_velocities =[r.target_qd_0,r.target_qd_1,r.target_qd_2,r.target_qd_3,r.target_qd_4,r.target_qd_5]
    joint_accelerations =[r.target_qdd_0,r.target_qdd_1,r.target_qdd_2,r.target_qdd_3,r.target_qdd_4,r.target_qdd_5]
    tool_speeds =[r.actual_TCP_speed_5]
print(tool_speeds)    

In [ ]:
#Jacobian relation between joint velocities and cartesian velocities
#Jacobian matrix of UR5 robot
J =np.array([[ 1.09215538e-01, -4.74888413e-01,  0.00000000e+00 , 0.00000000e+00,
   0.00000000e+00,  1.00000000e+00],
 [-3.30349865e-01,  0.00000000e+00, -4.74888413e-01,  1.00000000e+00,
   0.00000000e+00,  0.00000000e+00],
 [ 9.46500000e-02 , 0.00000000e+00, -4.74549974e-01,  1.00000000e+00,
   0.00000000e+00,  0.00000000e+00],
 [ 9.46500000e-02 , 0.00000000e+00, -8.22999739e-02,  1.00000000e+00,
   0.00000000e+00,  0.00000000e+00],
 [-6.55376883e-05,  8.22999739e-02,  1.16418321e-19,  0.00000000e+00,
   0.00000000e+00,  1.00000000e+00],
 [ 0.00000000e+00 , 0.00000000e+00 , 0.00000000e+00,  1.00000000e+00,
   0.00000000e+00, 0.00000000e+00]])

#xdot = J*qdot
i =0
tool_vel_array =[]
while(i<4500):
    joint_vel =[joint_values[0][i],joint_values[1][i],joint_values[2][i],joint_values[3][i],joint_values[4][i],joint_values[5][i]]
    tool_velocities = np.matmul(J,joint_vel)
    print(tool_velocities)
    tool_velocity=np.sqrt(math.pow(tool_velocities[0],2)+math.pow(tool_velocities[1],2)+math.pow(tool_velocities[2],2))
    tool_vel_array.append(tool_velocity)
    i = i+1
#print(tool_vel_array)

In [ ]:
velocity = 0.01
acceleration = 0.01
dt = 0.008
lookahead_time = 0.1
gain = 300

for i in range(4500):
    start = time.time()
    rtde_c.servoC(rtde_c.getForwardKinematics([joint_values[0][i],joint_values[1][i],joint_values[2][i],joint_values[3][i],joint_values[4][i],joint_values[5][i]]),0.01,0.01,0.01 )    
    end = time.time()
    duration = end - start
    if duration < dt:
        time.sleep(dt - duration)
rtde_c.servoStop()
rtde_c.stopScript()
    